# **Time Series**: Forecasting with Prophet and GluonTS

Source:  [https://github.com/d-insight/code-bank.git](https://github.com/d-insight/code-bank.git)  
License: [MIT License](https://opensource.org/licenses/MIT). See open source [license](LICENSE) in the Code Bank repository. 

-------------

## Overview

In this illustration, we compare two leading time series algorithms: __Prophet__ and __GluonTS__. 

  * __Prophet__ is a library developed by Facebook to forecast time series data based on an additive model, where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well. Paper available here: https://peerj.com/preprints/3190/

  * __GluonTS__ is a library developed by Amazone for deep-learning-based time series modeling. Paper available here: https://arxiv.org/pdf/1906.05264.pdf

For these illustrations, we use the dataset provided by Facebook Research: https://github.com/facebook/prophet

-------------

## **Part 0**: Setup

### Import packages

In [ ]:
# Import all packages
import pandas as pd 
import json
import datetime

# Time series packages
from fbprophet                        import Prophet
from fbprophet.diagnostics            import cross_validation, performance_metrics
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer                  import Trainer
from gluonts.evaluation.backtest      import make_evaluation_predictions
from gluonts.evaluation               import Evaluator
from gluonts.dataset.common           import ListDataset
from gluonts.dataset.field_names      import FieldName

# Plotting
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import warnings
warnings.filterwarnings('ignore')

## **Part 1**: Load and pre-process data

In [ ]:
# Load data
df = pd.read_csv('data/retail_sales.csv')
df['ds'] = pd.to_datetime(df['ds'])

print(df.shape)
df.head()

In [ ]:
# Plot the first 3 years
plt.plot(df['ds'].iloc[:37], df['y'].iloc[:37])
plt.xticks(df['ds'].iloc[:37])
plt.xlabel('Time')
plt.ylabel('Sales')
plt.xticks(rotation=90)
plt.grid()
plt.show()

## **Part 2**: Fit and evaluate Prophet model

In [ ]:
df

In [ ]:
%%time

# Fit model, make a 5 year forecast, and plot forecasts 
m = Prophet(seasonality_mode='additive', mcmc_samples=200, daily_seasonality=False, weekly_seasonality=False)
m.fit(df)
future = m.make_future_dataframe(periods=1825)
fcst = m.predict(future)
fig = m.plot(fcst)

In [ ]:
# Plot a subset of the above graph 
fig = m.plot(fcst)
ax = fig.gca()
ax.set_xlim([datetime.date(2015, 6, 1), datetime.date(2017, 6, 1)])
plt.show()

In [ ]:
fcst.tail()

In [ ]:
# Trend and yearly seasonality 
fcst = m.predict(future)
fig = m.plot_components(fcst)

Schema of how Prophet cross-validates data (this is a different dataset!).

<img src="https://facebook.github.io/prophet/static/diagnostics_files/diagnostics_3_0.png" width="700" height="700" align="center"/>

Image source: https://facebook.github.io/prophet/static/diagnostics_files/diagnostics_3_0.png

In [ ]:
# initial: 22.5 years = 8100 days
# horizon: last 2 years = 730 days
df_cv = cross_validation(m, initial='8100 days', horizon = '730 days')
df_cv.head()

In [ ]:
# Compute cross-validated performance metrics 
df_p = performance_metrics(df_cv)
df_p.tail(10)

## **Part 3**: Fit and evaluate GluonTS model

In [ ]:
# Split train and test data in the same way as above
df_train = df[df['ds'] < '2014-06-01'].copy()
df_test = df[df['ds'] >= '2014-06-01'].copy()

print('Train shape: {}'.format(df_train.shape))
print('Test shape: {}'.format(df_test.shape))

In [ ]:
df_ds = ListDataset([{FieldName.TARGET: df['y'], 
                    FieldName.START: df['ds'][0]}], 
                    freq='1D')

train_ds = ListDataset([{FieldName.TARGET: df_train['y'], 
                         FieldName.START: df_train['ds'][0]}], 
                        freq='1D')

test_ds = ListDataset([{FieldName.TARGET: df_test['y'], 
                         FieldName.START: df_test['ds'].values[0]}], 
                       freq='1D')

print('Original samples:'.ljust(20) + str(df_ds.calc_stats().num_time_observations))
print('Train samples:'.ljust(20) + str(train_ds.calc_stats().num_time_observations))
print('Test samples:'.ljust(20) + str(test_ds.calc_stats().num_time_observations))

In [ ]:
# Set up estimator 
estimator = SimpleFeedForwardEstimator(
    num_hidden_dimensions=[10],
    prediction_length=24,       # number of samples to predict -> "test set"
    context_length=100,
    freq='1D',
    trainer=Trainer(ctx="cpu",
                    epochs=100,
                    learning_rate=1e-3,
                    batch_size=32,
                    num_batches_per_epoch=50
                   )
)

In [ ]:
%%time 

# Train model
predictor = estimator.train(train_ds)


In [ ]:
# Evaluate the last "prediction_length" samples in the full dataset, trained on the training dataset 
forecast_it, ts_it = make_evaluation_predictions(
    dataset=df_ds,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation (24 months)
)

In [ ]:
forecasts = list(forecast_it)
tss = list(ts_it)

print('Forecast sample paths length:'.ljust(30) + str(len(forecasts[0].samples)))
print('Full time series length:'.ljust(30) + str(len(tss[0])))

In [ ]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))

In [ ]:
print(json.dumps(agg_metrics, indent=4))


## **Part 4**: Compare performance

We can interpret the RMSE and MAE in terms of dollar sales amounts and the MAPE as a percentage relative to the true sales figure. 

We find that the predictions are pretty accurate: the MAPE, for example, is around 3.9% (Prophet) and 2.3% (GluonTS). 

### FB Prophet

In [ ]:
df_p.tail()

### Amazon GluonTS

In [ ]:
print(json.dumps(agg_metrics, indent=4))


Amazon's GluonTS wins in terms of MSE, RMSE, and MAPE.